# Neural Networks

In this notebook, we will look at neural networks for image classification

In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchinfo import summary

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

In [3]:
trainsforms = transforms.Compose([
    transforms.ToTensor()
])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(train = True, root = './data', download = True, transform = transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dataset = torchvision.datasets.CIFAR10(train = False, root = './data', download = True, transform = transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False)
print(len(train_dataset), len(test_dataset))

Files already downloaded and verified
Files already downloaded and verified
50000 10000


In [5]:
model = torchvision.models.googlenet(weights = torchvision.models.GoogLeNet_Weights.DEFAULT)

In [6]:
model.fc = nn.Linear(1024, 10)
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [7]:
model = model.to(device)

In [8]:
random_tensor = torch.rand(10, 3, 32, 32).to(device)
print(summary(model = model, input_data=random_tensor, verbose=2, col_names=['input_size', 'output_size', 'num_params', 'trainable']))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
GoogLeNet                                [10, 3, 32, 32]           [10, 10]                  --                        True
├─BasicConv2d: 1-1                       [10, 3, 32, 32]           [10, 64, 16, 16]          --                        True
│    └─conv.weight                                                                           ├─9,408
│    └─bn.weight                                                                             ├─64
│    └─bn.bias                                                                               └─64
│    └─Conv2d: 2-1                       [10, 3, 32, 32]           [10, 64, 16, 16]          9,408                     True
│    │    └─weight                                                                           └─9,408
│    └─BatchNorm2d: 2-2                  [10, 64, 16, 16]          [10, 64, 16, 16]          128     

In [9]:
def train_step(model, loss_fn, optimizer, images, labels):
    model.train()
    outputs = model(images)
    logits = outputs.logits if isinstance(outputs, torchvision.models.GoogLeNetOutputs) else outputs
    loss = loss_fn(logits, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.0001)

In [11]:
for epoch in range(30):
    for batch in train_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        loss = train_step(model, loss_fn, optimizer, images, labels)
    print(f'Epoch [{epoch + 1}/30], Loss: {loss.item()}')

Epoch [1/30], Loss: 0.5446175336837769
Epoch [2/30], Loss: 1.1031750440597534
Epoch [3/30], Loss: 0.9356451034545898
Epoch [4/30], Loss: 0.13304513692855835
Epoch [5/30], Loss: 0.4555344581604004
Epoch [6/30], Loss: 0.2815956175327301
Epoch [7/30], Loss: 0.6705600023269653
Epoch [8/30], Loss: 0.6217655539512634
Epoch [9/30], Loss: 0.09291879087686539
Epoch [10/30], Loss: 0.5446655750274658
Epoch [11/30], Loss: 0.3005693852901459
Epoch [12/30], Loss: 0.0783669725060463
Epoch [13/30], Loss: 0.15164893865585327
Epoch [14/30], Loss: 0.36844319105148315
Epoch [15/30], Loss: 0.5716931223869324
Epoch [16/30], Loss: 0.01792934350669384
Epoch [17/30], Loss: 0.07769284397363663
Epoch [18/30], Loss: 0.004536966327577829
Epoch [19/30], Loss: 0.02278190478682518
Epoch [20/30], Loss: 0.10449191182851791
Epoch [21/30], Loss: 0.5532865524291992
Epoch [22/30], Loss: 0.007377977017313242
Epoch [23/30], Loss: 0.21821458637714386
Epoch [24/30], Loss: 0.14778362214565277
Epoch [25/30], Loss: 0.093803443014

In [12]:
accuracy = 0
model.eval()
with torch.inference_mode():
    for batch in test_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        logits = outputs.logits if isinstance(outputs, torchvision.models.GoogLeNetOutputs) else outputs
        _, predicted = torch.max(logits, 1)
        accuracy += (predicted == labels).sum().item()

print(f'Accuracy: {accuracy / len(test_dataset)}')

Accuracy: 0.8188
